In [1]:
# -*- coding: utf-8 -*-

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [115]:
import pandas as pd
import numpy as np
!pip install geopandas
import geopandas as gpd
import folium
from folium.plugins import HeatMap

# 1. 전유부

In [3]:
전유부 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/1.세종시_전유부.csv', encoding='CP949')
전유부.head(2)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,순번,대지위치,시군구코드,법정동코드,대지구분코드,번,지,대장구분코드명,건물명,특수지명,블록,로트,동명칭,호명칭,층구분코드명,층번호
0,1,세종특별자치시 반곡동 4121-1002번지,36110,10100,1,4121,1002,집합,세종펠리스,NaN,NaN,NaN,주건축물제1동,616,지상,6
1,2,세종특별자치시 반곡동 4121-1002번지,36110,10100,1,4121,1002,집합,세종펠리스,NaN,NaN,NaN,주건축물제1동,624,지상,6


## 변수 삭제

- 대지위치 : 시군구코드, 법정동코드, 번, 지 로 알 수 있음
- 대장구분코드명 : 모두 '집합'으로 같음
- 건물명 : 대지위치와 같은 이유
- 동명칭, 호명칭, 층번호 : 필요없음
- 층구분코드명 : 필요없을듯

In [4]:
전유부 = 전유부.drop(['대지위치','대장구분코드명','건물명','동명칭','호명칭','층구분코드명','층번호'], axis=1)
전유부.head(2)

,순번,시군구코드,법정동코드,대지구분코드,번,지,특수지명,블록,로트
0,1,36110,10100,1,4121,1002,NaN,NaN,NaN
1,2,36110,10100,1,4121,1002,NaN,NaN,NaN


## 결측값 처리

In [5]:
전유부.isnull().sum()

순번            0
시군구코드         0
법정동코드         0
대지구분코드        0
번             0
지             0
특수지명      15033
블록        15212
로트        38935
dtype: int64

In [6]:
전유부['특수지명'].unique()

array([nan, '4-1생활권 C1-6블럭', '4-1생활권', '행복도시 4-1생활권', '행정중심복합도시 4-1생활권',
       '행정중심복합도시4-1생활권', '3-3 생활권', '3-3생활권 C13-2', '3-3생활권',
       '세종특별자치시 3-3생활권', '행복중심복합도시3-3생활권', '행정중심복합도시 3-3생활권', '3-2생활권',
       '행정중심복합도시 3-2생활권', '행정중심복합도시3-2생활권', '행정중심복합도시', '3-1생활권',
       '행정중심복합도시 3-1생활권', '행정중심복합도시3-1생활권', '2-3생활권'], dtype=object)

In [7]:
전유부['블록'].unique()

array([nan, 'C 3-1', 'C 4-', 'C1-1BL', 'C1-4', 'C4-7블록', 'L2블록', 'M1블록',
       'M3블록', 'c4-6', 'C2-1', 'C2-2블록', 'L4블록', 'M2블록', 'L3블록', 'C14',
       'C10-2', 'C15', 'C16-1', 'C16-2BL', 'C5-4BL', 'C6', 'C8-1',
       'C8-2블록', 'C5-3블록', 'C12', 'C13-1', 'C13-3블럭', 'C16', 'M4블록',
       'M6블록', 'H1블록', 'M1', 'M3블럭', 'L1블럭', 'M5블록', '3-2생활권', 'M2BL',
       'M3BL', 'M4블럭', 'RC1-3'], dtype=object)

In [8]:
전유부['로트'].unique()

array([nan, '5', '1', '2', '-1', '-4', 'M4블록'], dtype=object)

### 결측 변수 삭제

- 블록 : 결측치가 많고, 특수지명과 일부 겹치는 것으로 보여 삭제
- 로트 : 대부분이 결측치

In [9]:
전유부 = 전유부.drop(['블록','로트'], axis=1)
전유부.head(2)

,순번,시군구코드,법정동코드,대지구분코드,번,지,특수지명
0,1,36110,10100,1,4121,1002,NaN
1,2,36110,10100,1,4121,1002,NaN


### 결측 변수 대체 (임의)

In [10]:
전유부['특수지명'].mode()  ### 최빈값

0    행정중심복합도시 3-3생활권
dtype: object

특수지명 변수의 결측값을 최빈값으로 대체. 임의로 대체한 것이므로 나중에 상의 후 변경

In [11]:
전유부['특수지명'] = 전유부['특수지명'].fillna('행정중심복합도시 3-3생활권')
전유부.head(2)

,순번,시군구코드,법정동코드,대지구분코드,번,지,특수지명
0,1,36110,10100,1,4121,1002,행정중심복합도시 3-3생활권
1,2,36110,10100,1,4121,1002,행정중심복합도시 3-3생활권


In [12]:
전유부.isnull().sum()

순번        0
시군구코드     0
법정동코드     0
대지구분코드    0
번         0
지         0
특수지명      0
dtype: int64

# 2. 표제부

In [13]:
표제부 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/2.세종시_표제부.csv', encoding='CP949')
표제부.head(2)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,순번,대지위치,시군구코드,법정동코드,대지구분코드,번,지,대장구분코드명,건물명,동명칭,연면적(㎡),구조코드명,주용도코드명,세대수(세대),사용승인일,Unnamed: 15
0,1,세종특별자치시 반곡동 146-75번지,36110,10100,1,146,75,일반,NaN,NaN,135.24,벽돌구조,단독주택,0,1977.0,NaN
1,2,세종특별자치시 반곡동 146-79번지,36110,10100,1,146,79,일반,NaN,1동,576,강파이프구조,동.식물관련시설,0,1999.0,NaN


## 변수 삭제

- 대지위치 : 시군구코드, 법정동코드, 번, 지 로 알 수 있음
- 대장구분코드명 : 의미 없어보여 임의로 삭제 -- 상의 후 결정
- 동명칭 : 필요없음
- 건물명 : 대지위치와 같은 이유
- Unnamed: 15 : 의미 알 수 없고 값이 한가지거나 결측값인 변수로, 삭제

In [14]:
표제부 = 표제부.drop(['대지위치','대장구분코드명','건물명','동명칭'], axis=1)
표제부.head(2)

,순번,시군구코드,법정동코드,대지구분코드,번,지,연면적(㎡),구조코드명,주용도코드명,세대수(세대),사용승인일,Unnamed: 15
0,1,36110,10100,1,146,75,135.24,벽돌구조,단독주택,0,1977.0,NaN
1,2,36110,10100,1,146,79,576,강파이프구조,동.식물관련시설,0,1999.0,NaN


In [15]:
표제부['Unnamed: 15'].unique()

array([      nan, 20130823.])

In [16]:
표제부['Unnamed: 15'].isnull().sum()

36728

In [17]:
표제부 = 표제부.drop(['Unnamed: 15'], axis=1)
표제부.head(2)

,순번,시군구코드,법정동코드,대지구분코드,번,지,연면적(㎡),구조코드명,주용도코드명,세대수(세대),사용승인일
0,1,36110,10100,1,146,75,135.24,벽돌구조,단독주택,0,1977.0
1,2,36110,10100,1,146,79,576,강파이프구조,동.식물관련시설,0,1999.0


## 결측값 처리

In [18]:
표제부.isnull().sum()

순번            0
시군구코드         0
법정동코드         0
대지구분코드        0
번             0
지             0
연면적(㎡)        0
구조코드명         9
주용도코드명       12
세대수(세대)       0
사용승인일      1212
dtype: int64

### 결측값 대체

- 구조코드명, 주용도코드명 : 최빈값으로 대체

In [19]:
print(표제부['구조코드명'].mode())
print(표제부['주용도코드명'].mode())

0    철근콘크리트구조
dtype: object
0    단독주택
dtype: object


In [20]:
표제부['구조코드명'] = 표제부['구조코드명'].fillna('철근콘크리트구조')
표제부['주용도코드명'] = 표제부['주용도코드명'].fillna('단독주택')
표제부.isnull().sum()

순번            0
시군구코드         0
법정동코드         0
대지구분코드        0
번             0
지             0
연면적(㎡)        0
구조코드명         0
주용도코드명        0
세대수(세대)       0
사용승인일      1212
dtype: int64

- 사용승인일 : 0으로 대체 (임의)

In [21]:
표제부['사용승인일'] = 표제부['사용승인일'].fillna(0)
표제부.isnull().sum()

순번         0
시군구코드      0
법정동코드      0
대지구분코드     0
번          0
지          0
연면적(㎡)     0
구조코드명      0
주용도코드명     0
세대수(세대)    0
사용승인일      0
dtype: int64

In [22]:
표제부.head(2)

,순번,시군구코드,법정동코드,대지구분코드,번,지,연면적(㎡),구조코드명,주용도코드명,세대수(세대),사용승인일
0,1,36110,10100,1,146,75,135.24,벽돌구조,단독주택,0,1977.0
1,2,36110,10100,1,146,79,576,강파이프구조,동.식물관련시설,0,1999.0


## 데이터 형식 변환

- 사용승인일 : 년도만 있으면 월일 0101로 임의 설정
- 법정동코드 : 세자리수로 뒤의 2자리 삭제. 임의 설정이므로 나중에 상의 후 변경



In [23]:
표제부 = 표제부.astype({'사용승인일':int})  ### 소수점 버림
표제부 = 표제부.astype({'사용승인일':str})
표제부.head()

,순번,시군구코드,법정동코드,대지구분코드,번,지,연면적(㎡),구조코드명,주용도코드명,세대수(세대),사용승인일
0,1,36110,10100,1,146,75,135.24,벽돌구조,단독주택,0,1977
1,2,36110,10100,1,146,79,576,강파이프구조,동.식물관련시설,0,1999
2,3,36110,10100,1,146,79,336,강파이프구조,동.식물관련시설,0,20041223
3,4,36110,10100,1,146,79,127.19,강파이프구조,창고시설,0,19950726
4,5,36110,10100,1,180,1,2697.74,철근콘크리트구조,교육연구시설,0,20190228


In [24]:
표제부['사용승인일'] = 표제부['사용승인일'].apply(lambda x: x+'0101' if len(x)==4 else x)
표제부.head()

,순번,시군구코드,법정동코드,대지구분코드,번,지,연면적(㎡),구조코드명,주용도코드명,세대수(세대),사용승인일
0,1,36110,10100,1,146,75,135.24,벽돌구조,단독주택,0,19770101
1,2,36110,10100,1,146,79,576,강파이프구조,동.식물관련시설,0,19990101
2,3,36110,10100,1,146,79,336,강파이프구조,동.식물관련시설,0,20041223
3,4,36110,10100,1,146,79,127.19,강파이프구조,창고시설,0,19950726
4,5,36110,10100,1,180,1,2697.74,철근콘크리트구조,교육연구시설,0,20190228


In [25]:
표제부.describe()

,순번,시군구코드,법정동코드,대지구분코드,번,지
count,36729.000000,36729.0,36729.000000,36729.000000,36729.000000,36729.000000
mean,18365.000000,36110.0,30262.805658,1.124234,310.429388,6.002968
std,10602.893355,0.0,8545.648928,0.472338,306.729458,48.020151
min,1.000000,36110.0,10100.000000,1.000000,0.000000,0.000000
25%,9183.000000,36110.0,25032.000000,1.000000,90.000000,0.000000
50%,18365.000000,36110.0,34022.000000,1.000000,226.000000,1.000000
75%,27547.000000,36110.0,36028.000000,1.000000,436.000000,5.000000
max,36729.000000,36110.0,39024.000000,3.000000,5477.000000,7007.000000


In [26]:
표제부['법정동코드'] = 표제부['법정동코드'].astype(str).str.slice(0,3)
표제부.head(2)

,순번,시군구코드,법정동코드,대지구분코드,번,지,연면적(㎡),구조코드명,주용도코드명,세대수(세대),사용승인일
0,1,36110,101,1,146,75,135.24,벽돌구조,단독주택,0,19770101
1,2,36110,101,1,146,79,576,강파이프구조,동.식물관련시설,0,19990101


# 3~10. 실거래가 데이터

In [27]:
아파트_매매 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/3.세종시_아파트(매매)_실거래가.csv', encoding= 'CP949')
연립다세대_매매 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/4.세종시_연립다세대(매매)_실거래가.csv', encoding='CP949')
단독다가구_매매 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/5.세종시_단독다가구(매매)_실거래가.csv', encoding='CP949')
오피스텔_매매 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/6.세종시_오피스텔(매매)_실거래가.csv', encoding='CP949')
아파트_전월세 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/7.세종시_아파트(전월세)_실거래가.csv', encoding='CP949')
연립다세대_전월세 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/8.세종시_연립다세대(전월세)_실거래가.csv', encoding='CP949')
단독다가구_전월세 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/9.세종시_단독다가구(전월세)_실거래가.csv', encoding='CP949')
오피스텔_전월세 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/10.세종시_오피스텔(전월세)_실거래가.csv', encoding='CP949')

In [28]:
아파트_매매.head(2)

,계약년월,계약일,시군구,본번,부번,단지명,전용면적(㎡),거래금액(만원),층,건축년도
0,201701,1,세종특별자치시 보람동,628,2,스타힐타워Ⅰ(C2-3),16.9849,8600,7,2016
1,201701,1,세종특별자치시 종촌동,683,0,가재마을2단지(호반베르디움3차),59.8652,27300,7,2014


In [29]:
print(list(아파트_매매.columns))
print(list(연립다세대_매매.columns))
print(list(단독다가구_매매.columns))
print(list(오피스텔_매매.columns))

['계약년월', '계약일', '시군구', '본번', '부번', '단지명', '전용면적(㎡)', '거래금액(만원)', '층', '건축년도']
['계약년월', '계약일', '시군구', '본번', '부번', '건물명', '전용면적(㎡)', '대지권면적(㎡)', '거래금액(만원)', '층', '건축년도']
['계약년월', '계약일', '시군구', '주택유형', '도로조건', '연면적(㎡)', '대지면적(㎡)', '거래금액(만원)', '건축년도']
['계약년월', '계약일', '시군구', '본번', '부번', '단지명', '전용면적(㎡)', '거래금액(만원)', '층', '건축년도']


## 3~6. 매매 데이터

- 주택유형 변수 추가 : 아파트 / 연립다세대 / 단독 / 다가구 / 오피스텔
- 계약날짜 변수 추가 : 계약년월 + 계약일 
- 계약년월, 계약일 변수 삭제
- 단독다가구의 대지면적 변수를 전용면적으로 rename

- 우선 단독다가구_매매의 건축년도 결측 대체

In [30]:
단독다가구_매매.isnull().sum()

계약년월        0
계약일         0
시군구         0
주택유형        0
도로조건        0
연면적(㎡)      0
대지면적(㎡)     0
거래금액(만원)    0
건축년도        4
dtype: int64

In [31]:
단독다가구_매매['건축년도'].mode()

0    2013.0
dtype: float64

In [32]:
단독다가구_매매['건축년도'] = 단독다가구_매매['건축년도'].fillna(2013)
단독다가구_매매['건축년도'].isnull().sum()

0

In [33]:
아파트_매매['주택유형'] = '아파트'
연립다세대_매매['주택유형'] = '연립다세대'
오피스텔_매매['주택유형'] = '오피스텔'
아파트_매매.head(2)

,계약년월,계약일,시군구,본번,부번,단지명,전용면적(㎡),거래금액(만원),층,건축년도,주택유형
0,201701,1,세종특별자치시 보람동,628,2,스타힐타워Ⅰ(C2-3),16.9849,8600,7,2016,아파트
1,201701,1,세종특별자치시 종촌동,683,0,가재마을2단지(호반베르디움3차),59.8652,27300,7,2014,아파트


In [34]:
def 계약날짜_추가(data):
    data['계약년월'] = data['계약년월'].astype(str)
    data['계약일'] = data['계약일'].astype('int64').apply(lambda x: "{:0>2d}".format(x))
    data['계약날짜'] = data[['계약년월','계약일']].apply(lambda x: ''.join(x), axis=1)

    data = data.drop(['계약년월','계약일'], axis=1)
    return data

아파트_매매 = 계약날짜_추가(아파트_매매)
연립다세대_매매 = 계약날짜_추가(연립다세대_매매)
단독다가구_매매 = 계약날짜_추가(단독다가구_매매)
오피스텔_매매 = 계약날짜_추가(오피스텔_매매)
아파트_매매.head(2)

,시군구,본번,부번,단지명,전용면적(㎡),거래금액(만원),층,건축년도,주택유형,계약날짜
0,세종특별자치시 보람동,628,2,스타힐타워Ⅰ(C2-3),16.9849,8600,7,2016,아파트,20170101
1,세종특별자치시 종촌동,683,0,가재마을2단지(호반베르디움3차),59.8652,27300,7,2014,아파트,20170101


In [35]:
단독다가구_매매 = 단독다가구_매매.rename(columns={'대지면적(㎡)':'전용면적(㎡)'})
단독다가구_매매.head(2)

,시군구,주택유형,도로조건,연면적(㎡),전용면적(㎡),거래금액(만원),건축년도,계약날짜
0,세종특별자치시 전의면 다방리,단독,8m미만,67.32,294.0,2000,1991.0,20170102
1,세종특별자치시 조치원읍 서창리,단독,8m미만,581.25,415.0,54500,1997.0,20170104


In [36]:
매매_실거래가 = pd.concat([아파트_매매, 연립다세대_매매, 단독다가구_매매, 오피스텔_매매])
매매_실거래가.tail(2)

,시군구,본번,부번,단지명,전용면적(㎡),거래금액(만원),층,건축년도,주택유형,계약날짜,건물명,대지권면적(㎡),도로조건,연면적(㎡)
266,세종특별자치시 어진동,0,0.0,푸르지오시티,23.74,8050,7.0,2014.0,오피스텔,20201005,NaN,NaN,NaN,NaN
267,세종특별자치시 어진동,0,0.0,2차푸르지오시티,21.92,7500,13.0,2014.0,오피스텔,20201006,NaN,NaN,NaN,NaN


## 7~10. 전월세 데이터

- 결측치 대체

In [37]:
print(list(아파트_전월세.columns))
print(list(연립다세대_전월세.columns))
print(list(단독다가구_전월세.columns))
print(list(오피스텔_전월세.columns))

['계약년월', '계약일', '시군구', '본번', '부번', '단지명', '전월세구분', '전용면적(㎡)', '보증금(만원)', '월세(만원)', '층', '건축년도']
['계약년월', '계약일', '시군구', '본번', '부번', '건물명', '전월세구분', '전용면적(㎡)', '보증금(만원)', '월세(만원)', '층', '건축년도']
['계약년월', '계약일', '시군구', '도로조건', '계약면적(㎡)', '전월세구분', '보증금(만원)', '월세(만원)', '건축년도']
['계약년월', '계약일', '시군구', '본번', '부번', '단지명', '전월세구분', '전용면적(㎡)', '보증금(만원)', '월세(만원)', '층', '건축년도']


In [38]:
단독다가구_전월세.isnull().sum()

계약년월         0
계약일          0
시군구          0
도로조건         0
계약면적(㎡)      0
전월세구분        0
보증금(만원)      0
월세(만원)       0
건축년도       210
dtype: int64

In [39]:
단독다가구_전월세['건축년도'].mode()

0    2013.0
dtype: float64

In [40]:
단독다가구_전월세['건축년도'] = 단독다가구_전월세['건축년도'].fillna(2013)
단독다가구_전월세['건축년도'].isnull().sum()

0

In [41]:
아파트_전월세['주택유형'] = '아파트'
연립다세대_전월세['주택유형'] = '연립다세대'
오피스텔_전월세['주택유형'] = '오피스텔'

아파트_전월세 = 계약날짜_추가(아파트_전월세)
연립다세대_전월세 = 계약날짜_추가(연립다세대_전월세)
단독다가구_전월세 = 계약날짜_추가(단독다가구_전월세)
오피스텔_전월세 = 계약날짜_추가(오피스텔_전월세)

전월세_실거래가 = pd.concat([아파트_전월세, 연립다세대_전월세, 단독다가구_전월세, 오피스텔_전월세])
전월세_실거래가.tail(2)

,시군구,본번,부번,단지명,전월세구분,전용면적(㎡),보증금(만원),월세(만원),층,건축년도,주택유형,계약날짜,건물명,도로조건,계약면적(㎡)
1209,세종특별자치시 도담동,246,5.0,세종한신휴시티,월세,24.03,500,40,6.0,2014.0,오피스텔,20201006,NaN,NaN,NaN
1210,세종특별자치시 나성동,713,0.0,세진이너스빌,월세,34.15,3000,35,6.0,2013.0,오피스텔,20201021,NaN,NaN,NaN


In [42]:
내일 할 거
1. 나머지 변수 전처리
2. 변수 타입 재정의 - 잘못된거
3. 각 데이터의 수치형 변수끼리 상관관계 확인 - 그래프 시각화 : scatterplot
4. 범주형-수치형 변수끼리 시각화 (범주형으로 groupby 하면 될듯) : boxplot, violinplot, barplot 등
5. 수치형 데이터와 geojson으로 지도 시각화

SyntaxError: ignored

# 11~13. 기타 실거래가 데이터

In [55]:
상업업무용_실거래가 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/11.세종시_상업업무용(매매)_실거래가.csv', encoding='CP949')
토지_실거래가 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/12.세종시_토지(매매)_실거래가.csv', encoding='CP949')
분양권_실거래가 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/13.세종시_분양권_실거래가.csv', encoding='CP949')
분양권_실거래가.head(2)

,계약년월,계약일,시군구,단지명,전용면적(㎡),거래금액(만원),층,분/입구분
0,201701,1,세종특별자치시 다정동,"가온마을 1단지 (2-1,힐스테이트)",74.9668,29250,5,분
1,201701,1,세종특별자치시 새롬동,새뜸마을 1단지(메이저시티),101.8063,39080,3,분


## 결측치 처리

### 상업업무용_실거래가

- 대지면적, 지분구분 변수 삭제
- 층 : 1로 대체
- 건축년도 : 2015로 대체

In [56]:
상업업무용_실거래가.isnull().sum()

계약년월            0
계약일             0
시군구             0
유형              0
도로명             0
용도지역            0
건축물주용도          0
도로조건            0
전용/연면적(㎡)       0
대지면적(㎡)       800
거래금액(만원)        0
층             452
지분구분         1070
건축년도            5
dtype: int64

In [57]:
상업업무용_실거래가.shape

(1070, 14)

In [58]:
상업업무용_실거래가['층'].mode()

0    1.0
dtype: float64

In [64]:
print(상업업무용_실거래가['건축년도'].mode(),'\n')
상업업무용_실거래가['건축년도'].describe()

0    2015.0
dtype: float64 



count    1065.000000
mean     2010.356808
std        12.777904
min      1924.000000
25%      2013.000000
50%      2015.000000
75%      2016.000000
max      2020.000000
Name: 건축년도, dtype: float64

In [65]:
상업업무용_실거래가 = 상업업무용_실거래가.drop(['대지면적(㎡)','지분구분'], axis=1)
상업업무용_실거래가['층'] = 상업업무용_실거래가['층'].fillna(1)
상업업무용_실거래가['건축년도'] = 상업업무용_실거래가['건축년도'].fillna(2015)
상업업무용_실거래가.isnull().sum()

계약년월         0
계약일          0
시군구          0
유형           0
도로명          0
용도지역         0
건축물주용도       0
도로조건         0
전용/연면적(㎡)    0
거래금액(만원)     0
층            0
건축년도         0
dtype: int64

### 토지_실거래가

In [66]:
토지_실거래가.isnull().sum()

계약년월        0
계약일         0
시군구         0
지목          0
용도지역        5
도로조건        0
계약면적(㎡)     0
거래금액(만원)    0
지분구분        0
dtype: int64

In [68]:
토지_실거래가['용도지역'].mode()

0    농림지역
dtype: object

In [69]:
토지_실거래가['용도지역'] = 토지_실거래가['용도지역'].fillna('농림지역')
토지_실거래가['용도지역'].isnull().sum()

0

### 분양권_실거래가

In [70]:
분양권_실거래가.isnull().sum()

계약년월        0
계약일         0
시군구         0
단지명         0
전용면적(㎡)     0
거래금액(만원)    0
층           0
분/입구분       0
dtype: int64

## 이상치 처리

### 수치형

In [72]:
상업업무용_실거래가.describe()

,계약년월,계약일,전용/연면적(㎡),거래금액(만원),층,건축년도
count,1070.000000,1070.000000,1070.000000,1.070000e+03,1070.000000,1070.000000
mean,201862.066355,17.034579,177.152813,5.938366e+04,4.928037,2010.378505
std,116.377013,8.689839,1411.453127,4.080682e+05,11.385476,12.751922
min,201701.000000,1.000000,3.330000,3.000000e+02,1.000000,1924.000000
25%,201712.000000,10.000000,46.230000,2.200000e+04,1.000000,2013.000000
50%,201901.000000,18.000000,80.370000,3.600000e+04,1.000000,2015.000000
75%,202003.000000,25.000000,130.560000,5.800000e+04,4.000000,2016.000000
max,202010.000000,31.000000,45615.690000,1.331200e+07,61.000000,2020.000000


In [73]:
토지_실거래가.describe()

,계약년월,계약일,계약면적(㎡),거래금액(만원)
count,32445.000000,32445.000000,32445.000000,3.244500e+04
mean,201859.827708,15.903498,977.360906,1.381544e+04
std,107.683247,8.837962,4529.831758,7.552323e+04
min,201701.000000,1.000000,1.000000,1.000000e+00
25%,201802.000000,8.000000,115.000000,1.007000e+03
50%,201812.000000,16.000000,330.000000,2.390000e+03
75%,201912.000000,23.000000,661.000000,1.120000e+04
max,202010.000000,31.000000,300893.000000,7.224946e+06


In [71]:
분양권_실거래가.describe()

,계약년월,계약일,전용면적(㎡),거래금액(만원),층
count,7432.000000,7432.000000,7432.000000,7432.000000,7432.000000
mean,201726.135361,15.981432,83.849784,38113.442546,10.022336
std,49.380618,8.647179,18.445517,8644.265951,6.515370
min,201701.000000,1.000000,20.850000,8000.000000,-1.000000
25%,201704.000000,9.000000,69.961000,32486.750000,5.000000
50%,201708.000000,16.000000,84.967600,38200.000000,9.000000
75%,201711.000000,23.000000,99.072500,43187.000000,14.000000
max,202010.000000,31.000000,209.185200,87394.000000,34.000000


In [79]:
분양권_실거래가['층'].value_counts()

 5     505
 6     490
 8     479
 3     475
 7     457
 4     453
 10    440
 2     429
 9     403
 11    396
 12    359
 13    321
 14    285
 1     279
 15    259
 16    250
 17    163
 18    146
 19    107
 20    104
 21     93
 23     88
 22     84
 25     77
 24     71
 26     54
 29     52
 27     47
 28     46
 30      6
 0       5
 34      3
 33      2
 32      2
-1       1
 31      1
Name: 층, dtype: int64

- 층의 0과 -1을 1로 변경

In [86]:
분양권_실거래가['층'] = 분양권_실거래가['층'].apply(lambda x: 1 if x in(-1,0) else x)
분양권_실거래가.describe()

,계약년월,계약일,전용면적(㎡),거래금액(만원),층
count,7432.000000,7432.000000,7432.000000,7432.000000,7432.000000
mean,201726.135361,15.981432,83.849784,38113.442546,10.023278
std,49.380618,8.647179,18.445517,8644.265951,6.513972
min,201701.000000,1.000000,20.850000,8000.000000,1.000000
25%,201704.000000,9.000000,69.961000,32486.750000,5.000000
50%,201708.000000,16.000000,84.967600,38200.000000,9.000000
75%,201711.000000,23.000000,99.072500,43187.000000,14.000000
max,202010.000000,31.000000,209.185200,87394.000000,34.000000


### 범주형

In [96]:
상업업무용_실거래가['도로조건'].value_counts()

25m미만    432
25m이상    354
12m미만    136
8m미만     115
-         33
Name: 도로조건, dtype: int64

In [99]:
토지_실거래가['도로조건'].value_counts()

-        17188
8m미만     12753
12m미만     2157
25m미만      326
25m이상       21
Name: 도로조건, dtype: int64

In [100]:
토지_실거래가['지분구분'].value_counts()

지분    19997
      12448
Name: 지분구분, dtype: int64

- 상업업무용의 도로조건 : 최빈값 25m미만으로 대체
- 토지의 도로조건 : 최빈값 8m미만으로 대체
- 토지의 지분구분 : 지분으로 대체 (변수 삭제해도 될듯)

In [110]:
상업업무용_실거래가['도로조건'] = 상업업무용_실거래가['도로조건'].apply(lambda x: '25m미만' if x=='-' else x)
토지_실거래가['도로조건'] = 토지_실거래가['도로조건'].apply(lambda x: '8m미만' if x=='-' else x)
토지_실거래가['지분구분'] = 토지_실거래가['지분구분'].apply(lambda x: '지분' if x==' ' else x)
토지_실거래가['지분구분'].value_counts()

지분    32445
Name: 지분구분, dtype: int64

# 14~15. 상권정보

In [120]:
상권정보 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/14.세종시_상권정보.csv', encoding='UTF8')
상권정보_업종코드 = pd.read_csv('/content/drive/Shareddrives/세종시_프로젝트/data/15.세종시_상권정보_업종코드.csv', encoding='UTF8')
상권정보.head(2)

,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,행정동코드,행정동명,도로명주소,lon,lat
0,D,소매,D21,철물/난방/건설자재소매,D21A06,철물/건설자재소매,G47511,철물 및 난방용구 소매업,3611025000,조치원읍,"세종특별자치시 조치원읍 새내로 90-1, (정리)",127.298552,36.599920
1,D,소매,D21,철물/난방/건설자재소매,D21A02,보일러/냉난방용품,G47511,철물 및 난방용구 소매업,3611037000,전의면,"세종특별자치시 전의면 왕의물로 40, (동교리)",127.204298,36.682526


In [129]:
상권정보.isnull().sum()

상권업종대분류코드      0
상권업종대분류명       0
상권업종중분류코드      0
상권업종중분류명       0
상권업종소분류코드      0
상권업종소분류명       0
표준산업분류코드     604
표준산업분류명      604
행정동코드          0
행정동명           0
도로명주소          0
lon            0
lat            0
dtype: int64

### 이건 그냥 map + heatmap 테스트

In [117]:
m = folium.Map(location=[36.57204094280581, 127.2666604789283], zoom_start=10)
hm_wide = HeatMap(list(zip(상권정보.lat.values, 상권정보.lon.values, 상권정보.행정동코드.values)),
                     min_opacity=0.2,
                     radius=5, blur=15,
                     max_zoom=1 
                 )
m.add_child(hm_wide)
m

In [132]:
상권정보.dtypes

상권업종대분류코드     object
상권업종대분류명      object
상권업종중분류코드     object
상권업종중분류명      object
상권업종소분류코드     object
상권업종소분류명      object
표준산업분류코드      object
표준산업분류명       object
행정동코드          int64
행정동명          object
도로명주소         object
lon          float64
lat          float64
dtype: object